# Normalizing a `Spectrum` to photometry

In [32]:
# Imports
from SEDkit import spectrum, synphot
import astropy.units as q
import astropy.table as at
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

Let's create a new `Spectrum` object by passing it arrays with astropy units

In [33]:
# Get the data from file
file = resource_filename('SEDkit', 'data/M0V_spectrum.txt')
wave, flux, unc = np.genfromtxt(file, unpack=True)
spec = spectrum.Spectrum(wave*q.um, flux*q.W/q.m**2/q.um, unc*q.W/q.m**2/q.um)

# Plot it!
fig = spec.plot
show(fig)

We can calculate a synthetic flux or magnitude in a given band by passing it a bandpass

In [34]:
jband = synphot.Bandpass('2MASS.J')
syn_flux = spec.synthetic_flux(jband)
syn_mag = spec.synthetic_magnitude(jband)

print('Jflux =',syn_flux)
print('Jmag =',syn_mag)

# Plot it!
fig = spec.plot
fig.circle([jband.eff.value], [syn_flux[0].value], color='red', size=15)
show(fig)

Jflux = (<Quantity 7.861487574504663e-12 W / (m2 um)>, <Quantity 1.981354797881429e-12 W / (m2 um)>)
Jmag = (6.4936121481998974, 0.27364142195017566)


A spectrum can also be normalized to given magnitude in a given band

In [35]:
old_jflux = spec.synthetic_flux(jband)
norm_j = spec.renormalize(16.332, jband)
new_jflux = norm_j.synthetic_flux(jband)

print('Old Jflx =',old_jflux)
print('New Jflux =',new_jflux)

# Plot it!
fig = norm_j.plot
fig.circle([jband.eff.value], [new_jflux[0].value], color='red', size=15)
show(fig)

Old Jflx = (<Quantity 7.861487574504663e-12 W / (m2 um)>, <Quantity 1.981354797881429e-12 W / (m2 um)>)
New Jflux = (<Quantity 1.0900740662453488e-18 W / (m2 um)>, <Quantity 2.7473471139686666e-19 W / (m2 um)>)


A spectrum can also be normalized to a table of photometry using a goodness-of-fit statistic.

In [36]:
# Make some dummy photometry
bands = ['2MASS.J','2MASS.H','2MASS.Ks']
mags = np.array([3.5e-18, 2.5e-18, 1e-18])*q.W/q.m**2/q.um
errs = np.array([3e-19, 2.5e-19, 2e-19])*q.W/q.m**2/q.um
bps = [synphot.Bandpass(b) for b in bands]
effs = np.array([i.eff.value for i in bps])*q.AA
photometry = at.QTable([bands, effs, mags, errs, bps], names=('band','eff','app_flux','app_flux_unc','bandpass'))

# Normalize to all magnitudes
norm_all = spec.norm_to_mags(photometry)

# And get the synthetic fluxes for comparison
syn_mags = [norm_all.synthetic_flux(b)[0].value for b in bps]

In [37]:
# Plot it
fig = norm_all.plot
fig.circle(effs.value, mags.value, color='black', size=15, legend='Input mags')
fig.circle(effs.value, syn_mags, color='red', size=15, legend='Synthetic mags')
show(fig)

Terrific!